**Function 8: High-dimensional Optimisation**

You’ve reach the final, 8-dimensional search space. High-dimensional black-box optimisation can be very difficult, so sticking to local solutions is not the worst idea here.


In [2]:
import numpy as np

from get_init_data import get_inputs, get_outputs

f_num = 8

X_init = get_inputs(f_num)
y_init = get_outputs(f_num)
# print(X_init)
# print(y_init)

In [3]:
from get_query_data import get_inputs, get_outputs

X_q = get_inputs(f_num)
y_q = get_outputs(f_num)
# print(X_q)
# print(y_q)

[0.431257, 0.895431]
[0.821456, 0.390248]
[0.19432, 0.682014, 0.011249]
[0.129382, 0.503612, 0.777124, 0.041672]
[0.964301, 0.306598, 0.758164, 0.118309]
[0.338127, 0.072341, 0.600298, 0.901134, 0.244792]
[0.120936, 0.893025, 0.516204, 0.032114, 0.74219, 0.28831]
[0.024893, 0.310587, 0.498236, 0.865901, 0.702134, 0.139804, 0.603781, 0.003172]
[0.431257, 0.895431]
[0.821456, 0.390248]
[0.19432, 0.682014, 0.011249]
[0.129382, 0.503612, 0.777124, 0.041672]
[0.964301, 0.306598, 0.758164, 0.118309]
[0.338127, 0.072341, 0.600298, 0.901134, 0.244792]
[0.120936, 0.893025, 0.516204, 0.032114, 0.74219, 0.28831]
[0.024893, 0.310587, 0.498236, 0.865901, 0.702134, 0.139804, 0.603781, 0.003172]
[0.557814, 0.91103]
[0.423839, 0.177067]
[0.939684, 0.438755, 0.426575]
[0.439915, 0.427314, 0.95773, 0.471256]
[0.297493, 0.25993, 0.547059, 0.410727]
[0.848123, 0.282866, 0.002497, 0.54845, 0.964839]
[0.24578, 0.13368, 0.704378, 0.80709, 0.748982, 0.40143]
[0.389062, 0.327444, 0.27934, 0.140607, 0.299915, 0

In [6]:
from get_init_data2 import get_inputs, get_outputs

X_init2 = get_inputs(f_num)
y_init2 = get_outputs(f_num)
# print(X_init2)
# print(y_init2)

In [8]:
X = np.concatenate((X_init, X_q, X_init2), axis=0)
y = np.concatenate((y_init, y_q, y_init2), axis=0)

print("Input : Output")
for i, v in enumerate(X):
    print(str(v) + ":" + str(y[i]))
# print("INPUTS")
# print(X)
# print("OUTPUTS")
# print(y)

Input : Output
[0.604994452514612 0.29221501955932727 0.9084527476191498
 0.35550624198849134 0.2016687192685087 0.575338005282615
 0.3103109513417007 0.7342813769904157]:7.398721101163708
[0.1780069588998261 0.5662226539934582 0.994861844649882
 0.21032500605287419 0.3201526568994505 0.7079087917386544
 0.635384488933402 0.1071316266051805]:7.005227361900734
[0.009076976680729043 0.8116261525080638 0.5205203597642096
 0.07568667521428252 0.2651118251498523 0.09165168939924984
 0.5924151454340054 0.36732026219027936]:8.45948161622808
[0.5060281642263985 0.6537301230139242 0.3634107791850698
 0.17798104853190577 0.09372830436573298 0.1974253305870869
 0.7558269002800702 0.29247233865659084]:8.284007811285548
[0.35990926424958747 0.24907567901155137 0.4959971701962457
 0.709214980573863 0.11498719465194007 0.28920692081786503
 0.5572951508023158 0.593881726256155]:8.606116791392116
[0.7788183442411031 0.0034194994840646142 0.3379831296954995
 0.5195277781785284 0.8209069931953403 0.53724

In [10]:
from ba_optimizer_v1 import BayesianOptimizer

optimizer = BayesianOptimizer(np.array(X, dtype=np.float64), y, bounds=(0, 1))

# Re-optimize with updated data
new_submission = optimizer.optimize_step(
    num_candidates=100000,
    acquisition_func='ei'
)

print(f"\nNEW RECOMMENDATION: {new_submission['best_point']}")
print(f"UCB Score: {new_submission['best_acquisition']:.4f}")

BAYESIAN OPTIMIZATION STEP
1. Computing function metrics...
   Current best: 9.7551
   Dataset size: 96
   Estimated noise: 0.0000

2. Updating surrogate model...
GP fitted with kernel: 3.16**2 * RBF(length_scale=2.16) + WhiteKernel(noise_level=1e-10)
Log-marginal likelihood: 63.4881

3. Generating 100000 candidate samples...
4. Computing EI acquisition function...
5. Selecting best points for submission...

RECOMMENDED NEXT POINT:
Location: [0.00993747 0.1751064  0.10512695 0.14295577 0.64804489 0.57103152
 0.33203552 0.76857967]
Acquisition: 0.1656
GP Prediction: 9.9206 ± 0.0494

NEW RECOMMENDATION: [0.00993747 0.1751064  0.10512695 0.14295577 0.64804489 0.57103152
 0.33203552 0.76857967]
UCB Score: 0.1656


In [11]:
####### Acquisition Function - Expected Improvement
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from scipy.stats import norm

kernel = C(1.0, (1e-3, 1e3)) * RBF(1.0, (1e-2, 1e2))

gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)
gpr.fit(X, y)

import itertools as it

x1 = np.linspace(0, 1, 8)

dim = 8
X_grid = np.fromiter(it.chain(*it.product(x1, repeat=dim)), dtype=float).reshape(-1,dim)
mean, std = gpr.predict(X_grid, return_std = True)

def compute_expected_improvement(x):
    mu, sigma = gpr.predict([x], return_std=True)
    f_best = np.max(y)
    sigma = sigma + 1e-9
    z = (mu - f_best) / sigma
    ei = (mu - f_best) * norm.cdf(z) + sigma * norm.pdf(z)
    return ei

ei_values = [compute_expected_improvement(x) for x in X_grid]

next_idx = np.argmax(ei_values)
next_query = X_grid[next_idx]

print("Expected Improvement - Next Query: ", next_query)

KeyboardInterrupt: 

In [19]:
from sklearn.gaussian_process.kernels import RBF, ConstantKernel

from ba_optimizer_v2 import BayesianOptimizer

optimizer_v2 = BayesianOptimizer(np.array(X, dtype=np.float64), y, bounds=(0, 1))

kernel = ConstantKernel(1.0, (1e-3, 1e3)) * RBF(1.0, (1e-2, 1e2))

new_submission = optimizer_v2.optimize_step(
    num_candidates=100000,
    acquisition_func='ei',
    kernel=kernel
)

print(f"\nNEW RECOMMENDATION: {new_submission['best_point']}")
print(f"UCB Score: {new_submission['best_acquisition']:.4f}")

BAYESIAN OPTIMIZATION STEP
1. Computing function metrics...
   Current best: 9.9499
   Dataset size: 97
   Estimated noise: 0.0000

2. Updating surrogate model...
GP fitted with kernel: 31.6**2 * RBF(length_scale=7.18)
Log-marginal likelihood: 120.9000

3. Generating 100000 candidate samples...
4. Computing EI acquisition function...
5. Selecting best points for submission...

RECOMMENDED NEXT POINT:
Location: [0.07706831 0.13456008 0.17534232 0.1296602  0.83185125 0.41171054
 0.14880121 0.02839497]
Acquisition: 0.0082
GP Prediction: 9.9413 ± 0.0302

NEW RECOMMENDATION: [0.07706831 0.13456008 0.17534232 0.1296602  0.83185125 0.41171054
 0.14880121 0.02839497]
UCB Score: 0.0082


# Calcualting after 27 May outputs

In [15]:
input_may_27 = np.array([0.046476, 0.122181, 0.201856, 0.066548, 0.77033 , 0.432087, 0.289013, 0.645225])
output_may_27 = np.float64(9.9499388871555)

X = np.append(X, np.array([input_may_27], dtype=np.float64), axis=0)
y = np.append(y, output_may_27)

y_max = np.max(y)
print("y_max is ", y_max, "at: ", X[np.where(y == y_max)][0])

y_max is  9.9499388871555 at:  [0.046476 0.122181 0.201856 0.066548 0.77033 0.432087 0.289013 0.645225]


In [22]:
optimizer_v2.add_observation(input_may_27, output_may_27)

optimizer_v2.optimize_step(
    num_candidates=10000,
    acquisition_func='ei',
    kernel=kernel)

Added observation: [0.046476 0.122181 0.201856 0.066548 0.77033  0.432087 0.289013 0.645225] -> 9.9499
BAYESIAN OPTIMIZATION STEP
1. Computing function metrics...
   Current best: 9.9499
   Dataset size: 98
   Estimated noise: 0.0000

2. Updating surrogate model...
GP fitted with kernel: 31.6**2 * RBF(length_scale=7.24)
Log-marginal likelihood: 132.9281

3. Generating 10000 candidate samples...
4. Computing EI acquisition function...
5. Selecting best points for submission...

RECOMMENDED NEXT POINT:
Location: [0.0367758  0.08940202 0.21038865 0.15722423 0.81773626 0.5591805
 0.10000647 0.84937986]
Acquisition: 0.0030
GP Prediction: 9.9413 ± 0.0160


{'best_point': array([0.0367758 , 0.08940202, 0.21038865, 0.15722423, 0.81773626,
        0.5591805 , 0.10000647, 0.84937986]),
 'best_acquisition': 0.002975457842422233,
 'top_k_points': array([[0.0367758 , 0.08940202, 0.21038865, 0.15722423, 0.81773626,
         0.5591805 , 0.10000647, 0.84937986],
        [0.18306279, 0.12509282, 0.13107799, 0.28875332, 0.65180096,
         0.61390037, 0.17910204, 0.70515769],
        [0.0388213 , 0.21132843, 0.09409961, 0.07459733, 0.8655805 ,
         0.69449439, 0.25434687, 0.98739604],
        [0.05276027, 0.0158827 , 0.02773352, 0.09330902, 0.91651986,
         0.64170149, 0.1258849 , 0.0562352 ],
        [0.06543302, 0.15169232, 0.214654  , 0.07572424, 0.9662545 ,
         0.75483435, 0.12374096, 0.00335353]]),
 'top_k_acquisitions': array([2.97545784e-03, 1.85935323e-03, 8.24097495e-04, 6.75437979e-04,
        2.74586099e-05]),
 'gp_predictions': (array([9.94129515, 9.94374431, 9.92204016, 9.87009589, 9.83948953]),
  array([0.01601392, 0.0106

In [25]:
####### Acquisition Function - Expected Improvement
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from scipy.stats import norm

kernel = C(1.0, (1e-3, 1e3)) * RBF(1.0, (1e-2, 1e2))

gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)
gpr.fit(X, y)

import itertools as it

x1 = np.linspace(0, 1, 8)

dim = 8
X_grid = np.fromiter(it.chain(*it.product(x1, repeat=dim)), dtype=float).reshape(-1,dim)
mean, std = gpr.predict(X_grid, return_std = True)

def compute_expected_improvement(x):
    mu, sigma = gpr.predict([x], return_std=True)
    f_best = np.max(y)
    z = (mu - f_best) / sigma
    ei = (mu - f_best) * norm.cdf(z) + sigma * norm.pdf(z)
    return ei

ei_values = [compute_expected_improvement(x) for x in X_grid]

next_idx = np.argmax(ei_values)
next_query = X_grid[next_idx]

print("Expected Improvement - Next Query: ", next_query)

Expected Improvement - Next Query:  [0.14285714 0.14285714 0.14285714 0.14285714 0.85714286 0.57142857
 0.14285714 0.57142857]


# Calcualting after 2 Jun outputs

In [27]:
input_jun_2 = np.array([0.036775, 0.089402, 0.210388, 0.157224, 0.817736, 0.55918 , 0.100006, 0.849379])
output_jun_2 = np.float64(9.9390180594539)

X = np.append(X, np.array([input_jun_2], dtype=np.float64), axis=0)
y = np.append(y, output_jun_2)

y_max = np.max(y)
print("Max y is ", y_max, "at: ", X[np.where(y == y_max)][0])

Max y is  9.9499388871555 at:  [0.046476 0.122181 0.201856 0.066548 0.77033 0.432087 0.289013 0.645225]


In [29]:
####### Acquisition Function - Expected Improvement
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from scipy.stats import norm

kernel = C(1.0, (1e-3, 1e3)) * RBF(1.0, (1e-2, 1e2))

gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)
gpr.fit(X, y)

import itertools as it

x1 = np.linspace(0, 1, 8)

dim = 8
X_grid = np.fromiter(it.chain(*it.product(x1, repeat=dim)), dtype=float).reshape(-1,dim)
mean, std = gpr.predict(X_grid, return_std = True)

def compute_expected_improvement(x):
    mu, sigma = gpr.predict([x], return_std=True)
    f_best = np.max(y)
    sigma = sigma + 1e-9
    z = (mu - f_best) / sigma
    ei = (mu - f_best) * norm.cdf(z) + sigma * norm.pdf(z)
    return ei

ei_values = [compute_expected_improvement(x) for x in X_grid]

next_idx = np.argmax(ei_values)
next_query = X_grid[next_idx]

print("Expected Improvement - Next Query: ", next_query)

Expected Improvement - Next Query:  [0.14285714 0.14285714 0.14285714 0.14285714 0.85714286 0.42857143
 0.14285714 0.57142857]


In [31]:
optimizer_v2.add_observation(input_jun_2, output_jun_2)

optimizer_v2.optimize_step(
    num_candidates=10000,
    acquisition_func='ei',
    kernel=kernel)

Added observation: [0.036775 0.089402 0.210388 0.157224 0.817736 0.55918  0.100006 0.849379] -> 9.9390
BAYESIAN OPTIMIZATION STEP
1. Computing function metrics...
   Current best: 9.9499
   Dataset size: 99
   Estimated noise: 0.0000

2. Updating surrogate model...
GP fitted with kernel: 31.6**2 * RBF(length_scale=7.31)
Log-marginal likelihood: 137.7851

3. Generating 10000 candidate samples...
4. Computing EI acquisition function...
5. Selecting best points for submission...

RECOMMENDED NEXT POINT:
Location: [0.18306279 0.12509282 0.13107799 0.28875332 0.65180096 0.61390037
 0.17910204 0.70515769]
Acquisition: 0.0008
GP Prediction: 9.9427 ± 0.0081


{'best_point': array([0.18306279, 0.12509282, 0.13107799, 0.28875332, 0.65180096,
        0.61390037, 0.17910204, 0.70515769]),
 'best_acquisition': 0.0008280899548669292,
 'top_k_points': array([[0.18306279, 0.12509282, 0.13107799, 0.28875332, 0.65180096,
         0.61390037, 0.17910204, 0.70515769],
        [0.05276027, 0.0158827 , 0.02773352, 0.09330902, 0.91651986,
         0.64170149, 0.1258849 , 0.0562352 ],
        [0.0388213 , 0.21132843, 0.09409961, 0.07459733, 0.8655805 ,
         0.69449439, 0.25434687, 0.98739604],
        [0.06543302, 0.15169232, 0.214654  , 0.07572424, 0.9662545 ,
         0.75483435, 0.12374096, 0.00335353],
        [0.14586147, 0.18529832, 0.043249  , 0.07517211, 0.51513066,
         0.43287122, 0.11285183, 0.02397994]]),
 'top_k_acquisitions': array([8.28089955e-04, 6.24069099e-04, 3.86030630e-05, 2.33119970e-05,
        1.10529033e-05]),
 'gp_predictions': (array([9.94272884, 9.87081591, 9.91974902, 9.8399212 , 9.86850787]),
  array([0.00809917, 0.043